In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as stats

import punct_utils as utils

plt.style.use("default")
plt.rcParams["text.usetex"] = False
np.random.seed(14159)

%load_ext autoreload
%autoreload 2

# Set parameters

In [ ]:
p = 500  # dimension
n = 2000  # sample size
L = 250  # band size

k = 2  # number of classes
pi = np.array([0.5, 0.5])  # proportion of each class
# centroid of each class
mu = stats.norm.rvs(size=(p, 1))
mu = 10 * mu / np.linalg.norm(mu)
M = np.concatenate([+mu, -mu], axis=1)


nbMC = 1  # number of Monte-Carlo simulations
n_eigvecs = 20  # number of computed eigenvectors

In [ ]:
c = p / n  # size ratio
beta = L / p

# size of each class
n_ = np.round(n * pi).astype(int)
n_[0] += n - np.sum(n_)

J = np.zeros((n, k), dtype=int)
sum_n = 0
for i, ni in enumerate(n_):
    J[sum_n : sum_n + ni, i] = 1
    sum_n += ni
np.random.shuffle(J)

setting = "$n = {} \quad c = {} \quad L/p = {}$".format(n, c, beta)

# Computations

In [ ]:
# Simulations
eigvals_t, eigvecs_t = utils.simul(
    nbMC, p, n, k, M, J, L, mask="toeplitz", comp=False, n_eigvecs=n_eigvecs
)

In [ ]:
M_wo = np.ones((p, 1))
J_wo = np.ones((n, 1))
eigvals_t_wo, eigvecs_t_wo = utils.simul(
    nbMC, p, n, k, M_wo, J_wo, L, mask="toeplitz", comp=False, n_eigvecs=n_eigvecs, noise=False
)

# Visualisation

In [ ]:
if k == 2:
    f, ax = plt.subplots(4,5, figsize=(12,8))
    for i in range(n_eigvecs):
        # Ensure that the first class has a greater mean component
        u_sgn_0 = np.sign(eigvecs_t_wo[0, i, J[:, 0] == 1].mean()-eigvecs_t_wo[0, i, J[:, 1] == 1].mean())
        u_sgn = np.sign(u_sgn_0*(eigvecs_t_wo[:, i, :]@eigvecs_t_wo[0, i, :]))
        u = eigvecs_t_wo[:, i, :]*u_sgn.reshape((nbMC, 1))
        # Plot
        ax[i//5,i%5].plot(u.mean(axis=0), marker='.', ls='')
        ax[i//5,i%5].grid(ls=':')
        ax[i//5,i%5].set_title("vep {}".format(i+1))
    f.suptitle('noiseless spike with toeplitz mask ' +setting)
    f.tight_layout()
    f.show() 
    
    f, ax = plt.subplots(4,5, figsize=(12,8))
    for i in range(n_eigvecs):
        # Ensure that the first class has a greater mean component
        u_sgn_0 = np.sign(eigvecs_t[0, i, J[:, 0] == 1].mean()-eigvecs_t[0, i, J[:, 1] == 1].mean())
        u_sgn = np.sign(u_sgn_0*(eigvecs_t[:, i, :]@eigvecs_t[0, i, :]))
        u = eigvecs_t[:, i, :]*u_sgn.reshape((nbMC, 1))
        # Plot
        ax[i//5,i%5].plot(u.mean(axis=0), marker='.', ls='')
        ax[i//5,i%5].grid(ls=':')
        ax[i//5,i%5].set_title("vep {}".format(i+1))
    f.suptitle('noise + spike with toeplitz mask ' +setting)
    f.tight_layout()
    f.show()

In [ ]:
import scipy.linalg as linalg

neigs_F = 20
neigs_U = 20

In [ ]:
F = eigvecs_t_wo[0, :neigs_F, :]
# F = linalg.dft(n, scale='sqrtn')[:neigs_F].real
U = eigvecs_t[0, :neigs_U, :]
print(U.shape)
u_1 = (np.eye(n) - (U.T) @ U) @ np.ones((n, 1)) / np.sqrt(n)
#U = np.concatenate((U, u_1.T), axis=0)
Pi_perp = np.eye(n) - U.T @ U
S = Pi_perp * (F.T @ F)

In [ ]:
j = J[:, 0] - J[:, 1]

In [ ]:
v, b = linalg.eigh(S, subset_by_index=[0, 0])
b = b.ravel()
j_est = np.sign(b.reshape(-1).real) * np.sign( j @ b.ravel() )
print(v)

score = min(np.sum(j_est != j), np.sum(j_est == j)) / n
mcr = min(score, 1 / 2 - score)
print('mcr == {}%'.format(score))

In [ ]:
plt.plot(b, ls="", marker=".")
plt.grid(ls=":")
plt.title(setting)
plt.ylim(-2 / np.sqrt(n), 2 / np.sqrt(n))
plt.show()